In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY
os.environ["GROQ_API_KEY"]=GROQ_API_KEY


In [5]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview")
llm.invoke("hii").content

'Hi there! How can I assist you today?'

In [6]:
message=[{
    "role":"system","content":"You are helpful content"},
    {"role":"user","content":"Hi how are you?"}]

In [7]:
result=llm.invoke(message)
print(result.content)

I'm doing well, thank you! How can I assist you today?


In [30]:
class Chatbot():
    def __init__(self,system=""):
        self.system = system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    
    def __call__(self,message):
        self.message.append({'role':'user',"content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
    def execute(self):
        llm=ChatGroq(model_name="Gemma2-9b-It")
        result=llm.invoke(self.message)
        return result.content

In [31]:
bot=Chatbot(system="You are helpful assistant")

In [32]:
bot("hi how are you?")

"I am doing well, thank you! As an AI, I don't have feelings, but I'm ready to assist you with any questions or tasks you may have. How can I help you today? 😊  \n\n"

In [33]:
bot.message

[{'role': 'system', 'content': 'You are helpful assistant'},
 {'role': 'user', 'content': 'hi how are you?'},
 {'role': 'assistant',
  'content': "I am doing well, thank you! As an AI, I don't have feelings, but I'm ready to assist you with any questions or tasks you may have. How can I help you today? 😊  \n\n"}]

In [42]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

Understand the Regular expression
Pattern Breakdown:

- ^: This matches the start of a string. It means the string must begin with what follows.
- Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
- (\w+):
The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
\w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".
- :: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
- (.*):
This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.
##### What does it do?
This regex is looking for a string that:

- Starts with the text "Action:".
Has a word right after it, separated by a colon.
- Then, after another colon, it captures everything that follows.

In [11]:
import re
action_re=re.compile("^Action: (\w+): (.*)")

In [38]:
# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

In [39]:
# Testing the pattern
match1 = action_re.match(text1)
match1

<re.Match object; span=(0, 19), match='Action: Move: North'>

In [40]:
match3 = action_re.match(text3)
match3

In [35]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [34]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [36]:
def calculate(number):
    return eval(number)

In [63]:
known_actions={
    "wikipedia":wikipedia,
    "calculate":calculate,
    "simon_blog_search":simon_blog_search
}

In [37]:
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result


In [43]:
prompt

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop your output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\n\nYour available actions are:\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point\nsyntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nsimon_blog_search:\ne.g. simon_blog_search: Python\nSearch Simon\'s blog for that term\n\nExample session:\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\nObservation: France is a country. The capital is Paris.\n\nYou then output:\nAnswer: The capital of France is Paris\n\nPlease Note: if you get 

In [44]:
bot=Chatbot(prompt)

In [45]:
bot("hi how are you?")

"Understood. I'm ready for my first question.  \n"

In [47]:
bot("what is your job?")

'Thought: I am an AI assistant designed to process information and respond to questions. \nAction: simon_blog_search: AI assistant\nPAUSE \n'

In [48]:
next_prompt="tell me about cryptocurrency from wikipedia"

In [55]:
result=bot(next_prompt)
result

'Thought: I will ask Wikipedia for a summary of cryptocurrency.\nAction: wikipedia: Cryptocurrency\nPAUSE '

In [62]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
actions

[<re.Match object; span=(0, 33), match='Action: wikipedia: Cryptocurrency'>]

In [57]:
action, action_input = actions[0].groups()

In [58]:
action

'wikipedia'

In [59]:
action_input

'Cryptocurrency'

In [60]:
known_actions[action]

<function __main__.wikipedia(query)>

In [64]:
known_actions[action](action_input)

'largest market for <span class="searchmatch">cryptocurrency</span>, declared all <span class="searchmatch">cryptocurrency</span> transactions illegal. This completed a crackdown on <span class="searchmatch">cryptocurrency</span> that had previously'

In [65]:
print(query("Fifteen * twenty five"))

Thought: Let's calculate the product of fifteen and twenty-five. 

Action: calculate: 15 * 25
PAUSE 

 -- running calculate 15 * 25
Observation: 375
Answer: Fifteen times twenty-five is 375.  

Answer: Fifteen times twenty-five is 375.  



In [66]:
query("Has Simon written about AI?")

Thought: I need to check Simon's blog for information about AI. 

Action: simon_blog_search: AI 
PAUSE  

 -- running simon_blog_search AI 
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lot 

'Thought:  Simon has written about AI and acknowledges the debate around the term. \n\nAnswer: Yes, Simon has written about AI. \n'